# **Homework Assignment No.2**
# **Osman Can SAVRAN** 
# **317199**

## 1) Reading the data

In [28]:
from SAVRAN_OSMANCAN_assignment_2 import *
import numpy as np
import pandas as pd
import sqlite3
import tempfile
import os, os.path
from pandasql import sqldf

Posts = pd.read_csv("travel_stackexchange/Posts.csv.gz", compression="gzip")
Comments = pd.read_csv("travel_stackexchange/Comments.csv.gz", compression="gzip")
PostLinks = pd.read_csv("travel_stackexchange/PostLinks.csv.gz", compression="gzip")
Users = pd.read_csv("travel_stackexchange/Users.csv.gz", compression="gzip")


## 2) Creation of the Database

In [ ]:


baza = os.path.join(tempfile.mkdtemp(), 'example.db')

if os.path.isfile(baza): # if this file already exists...
    os.remove(baza) # ...we will remove it

conn = sqlite3.connect(baza)

# import the data frame into the database
Posts.to_sql("Posts", conn)
Users.to_sql("Users", conn)
Comments.to_sql("Comments", conn)
PostLinks.to_sql("PostLinks", conn)

sql_result_1 = pd.read_sql_query("""SELECT Location, COUNT(*) AS Count
FROM (
SELECT Posts.OwnerUserId, Users.Id, Users.Location
FROM Users
JOIN Posts ON Users.Id = Posts.OwnerUserId
)
WHERE Location NOT IN ('')
GROUP BY Location
ORDER BY Count DESC
LIMIT 10""", conn)



sql_result_2 = pd.read_sql_query("""SELECT Posts.Title, RelatedTab.NumLinks
FROM
(
SELECT RelatedPostId AS PostId, COUNT(*) AS NumLinks
FROM PostLinks
GROUP BY RelatedPostId
) AS RelatedTab
JOIN Posts ON RelatedTab.PostId=Posts.Id
WHERE Posts.PostTypeId=1
ORDER BY NumLinks DESC""", conn)



sql_result_3 = pd.read_sql_query(f"""SELECT Title, CommentCount, ViewCount, CommentsTotalScore, DisplayName, Reputation, Location
FROM (
SELECT Posts.OwnerUserId, Posts.Title, Posts.CommentCount, Posts.ViewCount,
CmtTotScr.CommentsTotalScore
FROM (
SELECT PostId, SUM(Score) AS CommentsTotalScore
FROM Comments
GROUP BY PostId
) AS CmtTotScr
JOIN Posts ON Posts.Id = CmtTotScr.PostId
WHERE Posts.PostTypeId=1
) AS PostsBestComments
JOIN Users ON PostsBestComments.OwnerUserId = Users.Id
ORDER BY CommentsTotalScore DESC
""", conn)



sql_result_4 = pd.read_sql_query("""SELECT DisplayName, QuestionsNumber, AnswersNumber, Location, Reputation, UpVotes, DownVotes
FROM (
SELECT *
FROM (
SELECT COUNT(*) as AnswersNumber, OwnerUserId
FROM Posts
WHERE PostTypeId = 2
GROUP BY OwnerUserId
) AS Answers
JOIN
(
SELECT COUNT(*) as QuestionsNumber, OwnerUserId
FROM Posts
WHERE PostTypeId = 1
GROUP BY OwnerUserId
) AS Questions
ON Answers.OwnerUserId = Questions.OwnerUserId
WHERE AnswersNumber > QuestionsNumber
ORDER BY AnswersNumber DESC
LIMIT 5
) AS PostsCounts
JOIN Users
ON PostsCounts.OwnerUserId = Users.Id""", conn)

sql_result_5 = pd.read_sql_query("""
SELECT
Questions.Id,
Questions.Title,
BestAnswers.MaxScore,
Posts.Score AS AcceptedScore,
BestAnswers.MaxScore-Posts.Score AS Difference
FROM (
SELECT Id, ParentId, MAX(Score) AS MaxScore
FROM Posts
WHERE PostTypeId==2
GROUP BY ParentId
) AS BestAnswers
JOIN (
SELECT * FROM Posts
WHERE PostTypeId==1
) AS Questions
ON Questions.Id=BestAnswers.ParentId
JOIN Posts ON Questions.AcceptedAnswerId=Posts.Id
WHERE Difference>50
ORDER BY Difference DESC
""", conn)

conn.close()

                                               Title  CommentCount  ViewCount  \
0  Boss is asking for passport, but it has a stam...            24    61309.0   
1  How to intentionally get denied entry to the U...            37    72856.0   
2  Can I wear a bulletproof vest while traveling ...            26    23467.0   
3  OK we're all adults here, so really, how on ea...            29   113001.0   
4  Being separated from one's young children on a...            36    14308.0   

   CommentsTotalScore  DisplayName  Reputation                       Location  
0                 630        Megha        1679                           None  
1                 618    user11743        1997                           None  
2                 403       Ulkoma        9190         London, United Kingdom  
3                 388  hippietrail       75994  Sunshine Coast QLD, Australia  
4                 358  Emma-louise         611                           None  


## 3) Comparisons

### Query 1

In [ ]:
print(sql_result_1.equals(solution_1(Posts, Users)))


True


### Query 2

In [ ]:
sql_result_2.sort_values(by = sql_result_2.columns.tolist()).reset_index(drop = True).equals(solution_2(Posts, PostLinks).loc[:,sql_result_2.columns.tolist()].sort_values(by = sql_result_2.columns.tolist()).reset_index(drop = True))


True

### Query 3

I removed the LIMIT10 from the sql_query_read due to the high compilation time and added here for comparison

In [ ]:
sql_result_3.head(10).equals(solution_3(Posts, Users, Comments).head(10))

True

### Query 4

In [ ]:
print(sql_result_4.equals(solution_4(Posts, Users)))

True


### Query 5

In [ ]:
sql_result_5.sort_values(by = sql_result_5.columns.tolist()).reset_index(drop = True).equals(solution_5(Posts).loc[:,sql_result_5.columns.tolist()].sort_values(by = sql_result_5.columns.tolist()).reset_index(drop = True))

True